In [1]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
from dateutil.parser import parse

from datetime import timedelta, date
import numpy as np

In [2]:
countries = [
    'portugal','spain','italy','france','germany','uk',
    'us','iran','china','south-korea',
]

superes = {}

for country in countries:
    
    print (country)

    # Set the URL you want to webscrape from
    url = 'https://www.worldometers.info/coronavirus/country/%s'%country

    # Connect to the URL
    response = requests.get(url)

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text)

    res = pd.DataFrame()

    cases, deaths = [], []
    for ul_tag in soup.find_all('ul', {'class':'news_ul'})[:10]:
        i=0
        for li_tag in ul_tag.find_all('li', {'class':'news_li'}):
            i+=1
            if i==3: break
            strong_tag = li_tag.find_all('strong')
            cases += [strong_tag[0].text.split(' new case')[0]]
            deaths += [strong_tag[1].text.split(' new death')[0]]
    try:
        res['cases'] = [np.nan if (c.lower()==country or c in ['china','South Korea']) else int(c.replace(',','')) for c in cases]
    except:
        res['cases'] = [np.nan for c in cases]
    try:
        res['deaths'] = [np.nan if (d.lower()==country or d in ['china','South Korea']) else int(d.replace(',','')) for d in deaths]    
    except:
        res['deaths'] = [np.nan for c in deaths]

#     dates = []
#     for div_tag in soup.find_all('div'):
#         for d in div_tag:
#             if d.name == 'div' and d.get('class', '') == ['news_date']:
#                 dates += [d.text.split(' (')[0]]
    today = date.today()
    
    res['date'] = [str(d)[:10] 
                   for d in pd.to_datetime([pd.to_datetime(today)-timedelta(x+1)
                                            for x in range(len(cases))], 
                                            format='%Y%m%d')]
    print (res.set_index('date'))
    print ()
    
    superes[country] = res.set_index('date')

portugal
            cases  deaths
date                     
2020-12-22   2436      63
2020-12-21   2099      57
2020-12-20   3334      71
2020-12-19   3835      86
2020-12-18   4336      75
2020-12-17   4320      87
2020-12-16   4720      82

spain
            cases  deaths
date                     
2020-12-22   8544     260
2020-12-21   7503     112
2020-12-20  10419     149
2020-12-19   9772     181
2020-12-18   8822     195
2020-12-17   7614     388
2020-12-16   6417     129

italy
            cases  deaths
date                     
2020-12-22  13318     628
2020-12-21  10870     415
2020-12-20  15101     352
2020-12-19  16306     553
2020-12-18  17989     674
2020-12-17  18136     683
2020-12-16  17431     680

france
            cases  deaths
date                     
2020-12-22  11795     487
2020-12-21   5797     456
2020-12-20  12799     236
2020-12-19  17565     294
2020-12-18  15674     374
2020-12-17  18254     376
2020-12-16  17615     407

germany
            cases  death

In [3]:
xls = pd.ExcelFile("coronav.xls")
cases = xls.parse('cases')[['date']+countries]
deaths = xls.parse('deaths')[['date']+countries]
recovery = xls.parse('recovery')[['date']+countries]
cases['date'] = cases['date'].astype(str)
deaths['date'] = deaths['date'].astype(str)
recovery['date'] = recovery['date'].astype(str)

today = date.today()
lastdate = cases.date.iloc[-1]
iterations = (pd.to_datetime(today)-pd.to_datetime(lastdate)).days

writer = pd.ExcelWriter("coronav.xls", engine='xlsxwriter')
    
for tmp_sheet, name in zip([cases,deaths,recovery],
                           ['cases','deaths','recovery']):
    
    if name!='recovery':
    
        for i in range(1,iterations):
            print (i,name)
            tmp_date = pd.to_datetime(lastdate) + timedelta(days=i)
            tmp_country = [str(tmp_date)[:10]]
            print (tmp_country[0], end=' ')

            for country in countries:
                print (country, end=' ')

                if country not in ['china','south-korea']: 

                    try:
                        tmp_country += [superes[country].loc[tmp_country[0],name]]
                    except KeyError:
                        tmp_country += [np.nan]

                else:
                    tmp_country += [np.nan]

            print ('\n',tmp_country)

            tmp_sheet = tmp_sheet.append(
                pd.DataFrame([tmp_country],columns=['date']+countries)
            )

    tmp_sheet.to_excel(writer, name)

writer.save()

1 cases
2020-12-19 portugal spain italy france germany uk us iran china south-korea 
 ['2020-12-19', 3835, 9772, 16306, 17565, 23970, 27052, 198084, 6421, nan, nan]
2 cases
2020-12-20 portugal spain italy france germany uk us iran china south-korea 
 ['2020-12-20', 3334, 10419, 15101, 12799, 20822, 35928, 189463, 6312, nan, nan]
3 cases
2020-12-21 portugal spain italy france germany uk us iran china south-korea 
 ['2020-12-21', 2099, 7503, 10870, 5797, 19333, 33363, 201294, 6151, nan, nan]
4 cases
2020-12-22 portugal spain italy france germany uk us iran china south-korea 
 ['2020-12-22', 2436, 8544, 13318, 11795, 22495, 36804, 199080, 6208, nan, nan]
1 deaths
2020-12-19 portugal spain italy france germany uk us iran china south-korea 
 ['2020-12-19', 86, 181, 553, 294, 411, 534, 2678, 175, nan, nan]
2 deaths
2020-12-20 portugal spain italy france germany uk us iran china south-korea 
 ['2020-12-20', 71, 149, 352, 236, 350, 326, 1517, 177, nan, nan]
3 deaths
2020-12-21 portugal spain i

In [4]:
tmp_sheet

,date,portugal,spain,italy,france,germany,uk,us,iran,china,south-korea
0,2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,NaN
1,2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN
3,2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,2020-01-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
122,2020-05-24,NaN,NaN,1639.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,2020-05-25,NaN,NaN,1502.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,2020-05-26,NaN,NaN,2677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,2020-05-27,NaN,NaN,2443.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
